# Avlant

In [10]:
scipy.__version__

# 1.6.2 check.

'1.11.1'

In [36]:
import scipy
import numpy
import pandas
import os
lembas_path = '/nobackup/users/hmbaghda/Software/avlant_LEMBASGPU'
banList = []

filename = os.path.join(lembas_path, 'data', 'macrophage-Model.tsv')
net = pandas.read_csv(filename, sep='\t', index_col=False)
net = net[~ net["source"].isin(banList)]
net = net[~ net["target"].isin(banList)]

sources = list(net["source"])
targets = list(net["target"])
stimulation = numpy.array(net["stimulation"])
inhibition = numpy.array(net["inhibition"])
modeOfAction = 0.1 * numpy.ones(len(sources))  #ensuring that lack of known MOA does not imply lack of representation in scipy.sparse.find(A)
modeOfAction[stimulation==1] = 1
modeOfAction[inhibition==1] = -1
weights = modeOfAction
# networkList, nodeNames, weights = makeNetworkList(sources, targets, modeOfAction)  #0 == Target 1 == Source due to numpy sparse matrix structure
nodeNames = list(numpy.unique(sources + targets))
dictionary = dict(zip(nodeNames, list(range(len(nodeNames)))))
sourceNr = numpy.array([dictionary[x] for x in sources]) #colums
targetNr = numpy.array([dictionary[x] for x in targets]) #rows
size = len(nodeNames)
A_og = scipy.sparse.csr_matrix((weights, (sourceNr, targetNr)), shape=(size, size))
networkList_og = scipy.sparse.find(A_og)

# modeOfAction = numpy.array([[weights==1],[weights==-1]]).squeeze()

# Hratch

In [2]:
# prior knowledge signaling network
lembas_path = '/nobackup/users/hmbaghda/Software/avlant_LEMBASGPU'
import pandas as pd
import numpy as np
import scipy
import os
import sys
sclembas_path = '/home/hmbaghda/Projects/scLEMBAS/scLEMBAS'
sys.path.insert(1, os.path.join(sclembas_path))
from model.bionetwork import format_network

stimulation_label = 'stimulation'
inhibition_label = 'inhibition'
weight_label = 'mode_of_action'
source_label = 'source'
target_label = 'target'

net = pd.read_csv(os.path.join(lembas_path, 'data', 'macrophage-Model.tsv'), sep = '\t', index_col = False)

# ligand input and TF output
ligand_input = pd.read_csv(os.path.join(lembas_path, 'data', 'macrophage-Ligands.tsv'), sep='\t', low_memory=False, index_col=0)
tf_output = pd.read_csv(os.path.join(lembas_path, 'data', 'macrophage-TFs.tsv'), sep='\t', low_memory=False, index_col=0)

net = format_network(net, weight_label = weight_label, stimulation_label = stimulation_label, inhibition_label = inhibition_label)

ban_list = []

net = net[~ net[source_label].isin(ban_list)]
net = net[~ net[target_label].isin(ban_list)]

# create an edge list with node incides
node_labels = sorted(pd.concat([net[source_label], net[target_label]]).unique())
node_idx_map = {idx: node_name for node_name, idx in enumerate(node_labels)}

source_indices = net[source_label].map(node_idx_map).values
target_indices = net[target_label].map(node_idx_map).values

# # get edge list
# edge_list = np.array((target_indices, source_indices))
# edge_weights = net[weight_label].values
# get edge list *ordered by source-target node index*
n_nodes = len(node_labels)
A = scipy.sparse.csr_matrix((net[weight_label].values, (source_indices, target_indices)), shape=(n_nodes, n_nodes))
networkList = scipy.sparse.find(A)

In [3]:
np.all(A_og.toarray() == A.toarray())

True

In [4]:
networkList_og

(array([   0,    0,    1, ..., 1127, 1127, 1127], dtype=int32),
 array([596, 649, 732, ..., 443, 539, 623], dtype=int32),
 array([0.1, 0.1, 0.1, ..., 1. , 0.1, 0.1]))

In [5]:
for target_i in sourceNr:
    source_is = sourceNr[targetNr == target_i]
    source_is = np.array(source_is, dtype = np.int32)

In [6]:
source_is = sourceNr[targetNr == target_i]
# source_is.dtype = np.int32

In [7]:
np.equal(source_is, networkList_og[0][networkList_og[1] == target_i])

array([], dtype=bool)

In [8]:
node_i = 3
print(sourceNr[targetNr == node_i])
print(targetNr[sourceNr == node_i])
print(networkList_og[0][networkList_og[1] == node_i])
print(networkList_og[1][networkList_og[0] == node_i])

[]
[205 245]
[]
[205 245]


In [11]:
sourceNr

array([483, 792, 393, ...,  12,  13,  13])

In [13]:
networkList_og[0]

array([   0,    0,    1, ..., 1127, 1127, 1127], dtype=int32)

In [16]:
np.all(np.isin(networkList_og[1],sourceNr))

False

In [17]:
np.all(np.isin(networkList_og[0],sourceNr))

True

In [9]:
node_i = 3
print(source_indices[target_indices == node_i])
print(target_indices[source_indices == node_i])
print(networkList[0][networkList[1] == node_i])
print(networkList[1][networkList[0] == node_i])

[]
[205 245]
[]
[205 245]


In [22]:
source = np.array([0, 2, 4])
targets = np.array([1, 3, 3])
weights = np.array([0.1, -0.2, 1])
A = scipy.sparse.csr_matrix((weights, (source, targets)), shape=(5, 5))
netList = scipy.sparse.find(A)

In [23]:
A.todense()

matrix([[ 0. ,  0.1,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , -0.2,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  1. ,  0. ]])

In [53]:
netList

(array([0, 2, 4], dtype=int32),
 array([1, 3, 3], dtype=int32),
 array([ 0.1, -0.2,  1. ]))

In [27]:
networkList_og

(array([   0,    0,    1, ..., 1127, 1127, 1127], dtype=int32),
 array([596, 649, 732, ..., 443, 539, 623], dtype=int32),
 array([0.1, 0.1, 0.1, ..., 1. , 0.1, 0.1]))

In [29]:
dictionary

{'A5D8V6': 0,
 'A7MCY6': 1,
 'Ligand_GC': 2,
 'Ligand_IC': 3,
 'Ligand_IFNb': 4,
 'Ligand_IFNg': 5,
 'Ligand_IL10': 6,
 'Ligand_IL13': 7,
 'Ligand_IL4': 8,
 'Ligand_LPSc': 9,
 'Ligand_P3C': 10,
 'Ligand_PGE2': 11,
 'Ligand_TNFa': 12,
 'Ligand_upLPS': 13,
 'O00141': 14,
 'O00206': 15,
 'O00212': 16,
 'O00214': 17,
 'O00221': 18,
 'O00308': 19,
 'O00329': 20,
 'O00459': 21,
 'O00463': 22,
 'O00470': 23,
 'O00555': 24,
 'O00571': 25,
 'O14490': 26,
 'O14492': 27,
 'O14503': 28,
 'O14508': 29,
 'O14512': 30,
 'O14543': 31,
 'O14640': 32,
 'O14727': 33,
 'O14730': 34,
 'O14733': 35,
 'O14745': 36,
 'O14746': 37,
 'O14757': 38,
 'O14763': 39,
 'O14836': 40,
 'O14920': 41,
 'O14964': 42,
 'O14974': 43,
 'O14980': 44,
 'O15031': 45,
 'O15105': 46,
 'O15111': 47,
 'O15162': 48,
 'O15169': 49,
 'O15211': 50,
 'O15264': 51,
 'O15304': 52,
 'O15350': 53,
 'O15360': 54,
 'O15371': 55,
 'O15379': 56,
 'O15392': 57,
 'O15455': 58,
 'O15519': 59,
 'O15524': 60,
 'O15530': 61,
 'O15553': 62,
 'O43156':

In [45]:
node_name = 'Ligand_IC'
node_idx = dictionary[node_name]
net[net.source == node_name]

,source,target,direction,stimulation,inhibition,sources,references
5714,Ligand_IC,P08637,1,1,0,Immune complexes may activate FC receptors,NaN
5715,Ligand_IC,P12318,1,1,0,Immune complexes may activate FC receptors,NaN


In [52]:
back_map = {v:k for k,v in dictionary.items()}
test = networkList_og[1][networkList_og[0] == node_idx]
[back_map[target_idx] for target_idx in test]

['P08637', 'P12318']

In [51]:
scipy: 1.5.2

['P08637', 'P12318']

In [35]:
networkList_og

(array([   0,    0,    1, ..., 1127, 1127, 1127], dtype=int32),
 array([596, 649, 732, ..., 443, 539, 623], dtype=int32),
 array([0.1, 0.1, 0.1, ..., 1. , 0.1, 0.1]))